<a href="https://colab.research.google.com/github/giani-reyess/DataAnalysis_MAL/blob/main/MAL__DAipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Dependencies**

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats

In [ ]:
# Create df copy
df_anime = pd.read_csv('anime.csv', error_bad_lines=False,engine="python")
df_copy = df_anime.copy(deep=True)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 597: unexpected end of data


##**Fixing dataset for preprocessing**

### General cleaning

In [ ]:
# Slicing irrelevant data from dataset
df_copy = df_copy.iloc[0:,1:29]

# Drop irrelevant and repeated columns
df_copy.drop(['start_year', 'start_date', 'end_date', 'sfw'], axis=1, inplace=True)
df_copy.columns

### 'episode_duration' cleaning

In [ ]:
# Delete first 8 caracters for each value in from 'episode_duration' column 
df_copy['episode_duration'] = df_copy['episode_duration'].astype(str).apply(lambda x: x[8:])
df_copy.columns

### Cleaning multi-categorical data

In [ ]:
# Convert from strings to arrays

df_copy['genres'] = df_copy['genres'].apply(eval)
df_copy['themes'] = df_copy['themes'].apply(eval)
df_copy['genres'].values
df_copy['themes'].values

In [ ]:
# Count unique values

def count_uniques(series):
 return pd.Series([j for i in series for j in i])

unique_genres = count_uniques(df_copy['genres']).value_counts()
unique_themes = count_uniques(df_copy['themes']).value_counts()
unique_genres
unique_themes

In [ ]:
# Create a column for each genre where values will be boolean

def boolean_df(item_lists, unique_items):
# Create empty dict
    bool_dict = {}
    
    # Loop through all the tags
    for i, item in enumerate(unique_items):
        
        # Apply boolean mask
        bool_dict[item] = item_lists.apply(lambda x: item in x)
            
    # Return the results as a dataframe
    return pd.DataFrame(bool_dict) 

genre_bool = boolean_df(df_copy['genres'], unique_genres.keys())
themes_bool = boolean_df(df_copy['themes'], unique_themes.keys())
genre_bool
themes_bool
df_copy.head(2)

,title,type,score,scored_by,status,episodes,source,members,favorites,episode_duration,...,updated_at,start_season,real_start_date,real_end_date,broadcast_day,broadcast_time,genres,themes,demographics,studios
0,Fullmetal Alchemist: Brotherhood,tv,9.13,1871705,finished_airing,64.0,manga,2932347,204645,0:24:20,...,2022-04-18 05:06:13+00:00,spring,2009-04-05,2010-07-04,sunday,17:00:00,"[Action, Adventure, Drama, Fantasy]",[Military],['Shounen'],['Bones']
1,Hunter x Hunter (2011),tv,9.04,1509622,finished_airing,148.0,manga,2418883,185178,0:23:37,...,2022-04-18 05:11:21+00:00,fall,2011-10-02,2014-09-24,sunday,10:55:00,"[Action, Adventure, Fantasy]",[],['Shounen'],['Madhouse']


##**Preprocessing**

###Categoriacal variables 

In [ ]:
categ_var_encoded = pd.get_dummies(df_copy[['type','title','status','source', 'start_season', 'broadcast_day','real_start_date', 'real_end_date', 'broadcast_time']])
categ_var_encoded

,type_movie,type_music,type_ona,type_ova,type_special,type_tv,title_3-gatsu no Lion,title_3-gatsu no Lion 2nd Season,title_5-toubun no Hanayome,title_5-toubun no Hanayome ∬,...,broadcast_time_21:00:00,broadcast_time_21:30:00,broadcast_time_22:00:00,broadcast_time_22:30:00,broadcast_time_22:50:00,broadcast_time_23:00:00,broadcast_time_23:15:00,broadcast_time_23:30:00,broadcast_time_23:45:00,broadcast_time_23:53:00
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
591,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
592,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
593,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


###Numerical variables

In [ ]:
num_var_scaled = stats.zscore(df_copy[['score','scored_by','episodes','members','favorites']])
num_var_scaled

,score,scored_by,episodes,members,favorites
0,2.779939,4.177998,NaN,4.239152,8.536614
1,2.518977,3.176428,NaN,3.292119,7.671024
2,2.605964,2.678186,NaN,2.301400,1.746266
3,2.634960,2.464602,NaN,3.015897,7.133448
4,2.258015,2.869348,NaN,2.521993,2.880110
...,...,...,...,...,...
590,-0.467587,-0.804625,NaN,-0.898271,-0.555171
591,-0.206625,-0.823067,NaN,-0.945893,-0.547434
592,-2.033359,2.268859,NaN,2.273061,0.614556
593,-1.337460,-0.631836,NaN,-0.793973,-0.524046


In [32]:
!git remote set-url origin https://github.com/giani-reyess/DataAnalysis_MAL.git

In [33]:
!git push origin

fatal: The current branch master has no upstream branch.
To push the current branch and set the remote as upstream, use

    git push --set-upstream origin master

